In [87]:
import googlemaps
import folium
import pandas as pd
import numpy as np
import heapq
from haversine import haversine #거리구하기
from sklearn.preprocessing import minmax_scale #정규화

##1. data 불러오기
total_df = pd.read_csv("../all_df.csv", encoding = "EUC-KR" )
df = total_df.rename(index=str, columns={"Unnamed: 0" : "ID"})
df = df.drop(columns = ['경유여부','거리'])

##2. 출발지, 도착지
sp = (37.506059, 127.036863) #출발지 37.506059, 127.036863
ep = (37.509122, 127.043816) #도착지 37.509122, 127.043816
radius = 100
#4.중복값 제거
df = df.join( df.groupby("명칭")["ID"].count(), on = '명칭', rsuffix = '_cnt')
df = df[~(df.ID_cnt > 1)].sort_values(by = 'ID')
df = df.drop(columns = ['ID_cnt'])

#밝기 max
l_max = df['총밝기'].max()

# 3.table에 SP, EP정보 저장
df = df.append(df.from_dict({'ID' : -1,'point_ID':'Point_SP', '위도':sp[0], '경도':sp[1], '밝기':-1
                             , '분류':'출발지', '명칭':'출발지', '총밝기' : -1 }, orient = 'index').T)

df = df.append(df.from_dict({'ID' : len(total_df), 'point_ID':'Point_EP', '위도':ep[0], '경도':ep[1], '밝기': 999
                             , '분류':'도착지', '명칭':'도착지', '총밝기' : 999 }, orient = 'index').T)



df = df.sort_values(by='ID').reset_index(drop=True)
df1 = df
df1['ID'] = df1.index
df1.tail(4)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
1444,1444,Point_2087,127.043,역삼동 700-13,25,CCTV,37.505,70
1445,1445,Point_2088,127.037,역삼동 652-5,25,CCTV,37.5082,50
1446,1446,Point_2089,127.034,역삼동 735-18,25,CCTV,37.4979,110
1447,1447,Point_EP,127.044,도착지,999,도착지,37.5091,999


# 길찾기 구현 시작 (A* 알고리즘)
> 참고 사이트 : https://codeday.me/ko/qa/20190421/370160.html
### 1. 초기화

In [83]:
#!pip install geopy
import csv, geopy.distance, collections, heapq
##name
Location = collections.namedtuple("Location", "ID point_ID 분류 명칭 위도 경도 밝기 총밝기".split())
data = {}

for i in range(0,len(df1)):
        data[i] = Location(  df1.loc[i,"ID"]
                           , df1.loc[i,"point_ID"]
                           , df1.loc[i,"분류"]
                           , df1.loc[i,"명칭"]
                           , df1.loc[i,"위도"]
                           , df1.loc[i,"경도"]
                           , df1.loc[i,"밝기"]
                           , df1.loc[i,"총밝기"])
        
print( type(data) , len(data))
#sorted(data.values(), key=lambda x: calcH(0,len(data)-1))[0:4]

startIndex = 0                                                   #출발지
endIndex =len(data) - 1                                         #도착지
center_index = 0
h = calcH(startIndex, endIndex)                                    #도착지까지의 총거리
g = 0
f = h
#heap으로 하면 좋겠지만, sclaedF 를 구하기 위해 dataFrame으로 
openlist = pd.DataFrame(columns = "ID F G H L parentID".split() )
openlist.loc[0] = [int(startIndex), h, 0, h, 0 , None]                #openlist 초기화

closedlist = {}      # map visited nodes to parent #LinkedList



<class 'dict'> 1448


In [84]:
import copy
#heap으로 하면 좋겠지만, sclaedF 를 구하기 위해 dataFrame으로 
openlist = pd.DataFrame(columns = "ID F G H L W parentID".split() )
openlist.loc[0] = [startIndex, h, 0, h, 0 ,0, None] 

items = getneighbors(center_node.ID)
item = openlist.loc[0]

center_node = item
print(center_node , "\n ", item, len(items))


ID            0.000000
F           701.535109
G             0.000000
H           701.535109
L             0.000000
W             0.000000
parentID           NaN
Name: 0, dtype: float64 
  ID            0.000000
F           701.535109
G             0.000000
H           701.535109
L             0.000000
W             0.000000
parentID           NaN
Name: 0, dtype: float64 10


In [85]:
#거리 구하기
def calcH(start_index, end_index):
    distance = haversine([data[start_index].위도, data[start_index].경도] , [data[end_index].위도, data[end_index].경도] ,  unit='m') 
    return distance

def get_f(center_node, item ):
    g = center_node.G + calcH(center_node.ID, item.ID)
    h = calcH(item.ID, endIndex)
    f = g + h   
    #sf = get_scaled_f(item,f,g,h)
    return f

def get_scaled_f(item, f, g, h):
    openlist_tmp =copy.deepcopy(openlist)
    openlist_tmp = openlist
    g = center_node.G + calcH(center_node.ID, item.ID)
    h = calcH(item.ID, endIndex)
    f = g + h   

#3. 밝기 이용한 가중치 추가 W(weight = 4F(거리) + 6L(밝기))
def get_w(df) : 
    w = np.array(df.apply(lambda x : (x['총밝기'] * 45 ) + ( x['scaled_F'] * 100) , axis = 1)).reshape(-1,1)
    w = minmax_scale(w, copy=True, feature_range =(0,l_max))
    return w


#주변 점들 구하기
def getneighbors(center_index, n=10):
    rad = radius
#     while True :
#         res = df [ df.apply( lambda x : haversine(point, [x['위도'], x['경도']] ,  unit='m') , axis = 1  ) < rad ]
#         res = res[ ~res['point_ID'].isin(closed_list_id) ] # 1-2 . 이미 선택했던 point들 제외   
#         if (len(res) == 0  ) : 
#             rad += 10
#         else : 
#             break
    
    while True:
        res = data.values(), key=lambda x: calcH(center_index, x.ID)
        if len(res) ==0 :
            
        sorted(data.values(), key=lambda x: calcH(center_index, x.ID))[1:n+1]
    
    
    ##data로 출력
    return sorted(data.values(), key=lambda x: calcH(center_index, x.ID))[1:n+1]

#경로 추출하기
def getParent(closedlist, index):
    path = []
    while index is not None:
        path.append(index)
        index = closedlist.get(index, None)
    return [data[i] for i in path[::-1]]

In [86]:
#get_f(center_node, item)
items = getneighbors(center_index, n=10)
item = items[0]
get_f(center_node, item)


714.0372119237594

In [75]:
fin = []

while len(openlist) >= 1:
    # 우선순위큐 방식, 가장 점수높은 후보 꺼내서 중심점으로 만들기
    openlist.sort_values(by="W")
    center_node = openlist.loc[0]
    center_ID = int(center_node.ID)

    # 이미 방문한 지점인지 검사
     if center_ID in closedlist:
         continue
        
    closedlist[center_ID] = center_node.parentID

     # 만약 도착지라면 종료하고, 경로 추출하기
    if center_ID == endIndex:
         print("Complete")
        for p in getParent(closedlist, center_ID) :
            fin.append(p)
        break
        
    # 주변 점들을 구하고, 점들마다 예상 가중치를 쭉 구해온다.
    items = getneighbors(center_ID)
    
    
#     for item in items :
#         f = get_f(center_node, item )
        

        
        
        #현재점을 거쳐서 도착지까지 g 계산
#         for i in range(0, len(openlist)):
#             ids.append(openlist[i][1].ID)
#         if other.ID in ids

        ####openList에 추가###
        #새롭게 계산한 예상되는 F  < 이미 openlist에 있는 F  : 새롭게 예상한 아이가 더 좋은 경로 -> parentID 갱신
        # but, 중복되게 들어가더라도 항상 가장 가능성 높은 아이가 자동으로 나오므로 굳이 갱신할 필요가없다.
        #새롭게 계산한 예상되는 F  > 이미 openlist에 있는 F : 그대로 유지  



IndentationError: unexpected indent (<ipython-input-75-0998ca965c6a>, line 4)

In [74]:
openlist

    

0.0